In [1]:
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
import seaborn as sns
from pathlib import Path
from IPython.display import display, Markdown

In [2]:
def print_files(data_path: Path):
    files = data_path.glob('*.csv')
    for file in files:
        print(file.name)

In [3]:
data_path = Path('../data/')
print_files(data_path)

gas_prices.csv
client.csv
forecast_weather.csv
historical_weather.csv
weather_station_to_county_mapping.csv
train.csv
electricity_prices.csv


# Carga de los datos

In [4]:
def load_data(data_path: Path) -> dict[str, pl.DataFrame]:
    files = data_path.glob('*.csv')
    data_dict = dict()

    for file in files:
        file_name = file.stem
        file_path = file.resolve()

        if 'weather_station' in file_name:
            file_name = 'weather_station'
        
        data_dict[file_name] = pl.read_csv(file_path, try_parse_dates=True)
    
    return data_dict

In [5]:
data_dict = load_data(data_path)

# Descripción de los datos

In [6]:
def print_categories(df: pl.DataFrame, cat_cols: list[str]):
    cols = list(set(cat_cols).intersection(df.columns))

    display(Markdown('#### Categorías:'))
    if cols:   
        for col in cols:
            display(Markdown(f'{col}:'))
            print(df[col].unique().to_list())
    else:
        display(Markdown('Sin columnas categóricas.'))


In [7]:
def print_correlations(df: pl.DataFrame, exclude: set[list]):
    display(Markdown('Correlations:'))
    display(
        df.select(
            pl.col(pl.Int64, pl.Float64).exclude(categorical_columns)
        ).corr().to_pandas()
    )

In [8]:
categorical_columns = [
    'county',
    'product_type',
    'is_business',
    'county_name',
    'is_consumption'
]

In [9]:
# Display DataFrames
for table in data_dict.keys():
    tmp_df = data_dict[table]
    # Data visualization
    display(Markdown(f'### **{table}:**'))
    display(tmp_df.head())
    display(Markdown(f'shape: {tmp_df.shape}'))

    # Data Statistics
    display(Markdown('#### Estadísticas:'))
    display(tmp_df.describe())

    # Categorical columns
    print_categories(tmp_df, categorical_columns)

    # Correlations
    print_correlations(tmp_df, categorical_columns)

### **gas_prices:**

forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
date,f64,f64,date,i64
2021-09-01,45.23,46.32,2021-08-31,1
2021-09-02,45.62,46.29,2021-09-01,2
2021-09-03,45.85,46.4,2021-09-02,3
2021-09-04,46.3,46.8,2021-09-03,4
2021-09-05,46.3,46.58,2021-09-04,5


shape: (637, 5)

#### Estadísticas:

statistic,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
str,str,f64,f64,str,f64
"""count""","""637""",637.0,637.0,"""637""",637.0
"""null_count""","""0""",0.0,0.0,"""0""",0.0
"""mean""","""2022-07-16 00:00:00""",95.03675,107.754631,"""2022-07-15 00:00:00""",319.0
"""std""",null,47.552295,54.743666,null,184.030342
"""min""","""2021-09-01""",28.1,34.0,"""2021-08-31""",1.0
"""25%""","""2022-02-07""",60.0,67.53,"""2022-02-06""",160.0
"""50%""","""2022-07-16""",85.21,93.47,"""2022-07-15""",319.0
"""75%""","""2022-12-22""",109.0,130.74,"""2022-12-21""",478.0
"""max""","""2023-05-30""",250.0,305.0,"""2023-05-29""",637.0


...

statistic,forecast_date,euros_per_mwh,origin_date,data_block_id
str,str,f64,str,f64
"""count""","""15286""",15286.0,"""15286""",15286.0
"""null_count""","""0""",0.0,"""0""",0.0
"""mean""","""2022-07-16 11:16:41.857909""",157.064176,"""2022-07-15 11:16:41.857909""",318.99071
"""std""",null,121.148625,null,183.890301
"""min""","""2021-09-01 00:00:00""",-10.06,"""2021-08-31 00:00:00""",1.0
"""25%""","""2022-02-07 05:00:00""",85.29,"""2022-02-06 05:00:00""",160.0
"""50%""","""2022-07-16 12:00:00""",128.28,"""2022-07-15 12:00:00""",319.0
"""75%""","""2022-12-22 17:00:00""",199.8,"""2022-12-21 17:00:00""",478.0
"""max""","""2023-05-30 23:00:00""",4000.0,"""2023-05-29 23:00:00""",637.0


#### Categorías:

Sin columnas categóricas.

Correlations:

,euros_per_mwh,data_block_id
0,1.000000,-0.026326
1,-0.026326,1.000000


In [21]:
client_1 = data_dict["train"].filter(
    (pl.col('client_id') == 1) & 
    (pl.col('is_consumption') == 1) &
    (pl.col('year') == 2022) &
    (pl.col('month') == 1)
)
sns.scatterplot(x=client_1['datetime'], y=client_1['temperature_hw'])

ColumnNotFoundError: unable to find column "client_id"; valid columns: ["county", "is_business", "product_type", "target", "is_consumption", "datetime", "data_block_id", "row_id", "prediction_unit_id"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["county", "is_business", "product_type", "target", ...]; PROJECT */9 COLUMNS